In [ ]:
# 1 initiation

In [ ]:
%pwd

In [ ]:
import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data/red'

In [ ]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils import *
from vgg16 import Vgg16
%matplotlib inline

In [ ]:
# 2 structure the data

In [ ]:
cd $DATA_HOME_DIR
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown

In [ ]:
%cd $DATA_HOME_DIR/train

In [ ]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(2000): os.rename(shuf[i], DATA_HOME_DIR+'/valid/' + shuf[i])

In [ ]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(200): copyfile(shuf[i], DATA_HOME_DIR+'/sample/train/' + shuf[i])

In [ ]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(50): copyfile(shuf[i], DATA_HOME_DIR+'/sample/valid/' + shuf[i])

In [ ]:
# 3 train

In [ ]:
%cd $DATA_HOME_DIR

In [ ]:
path = DATA_HOME_DIR + '/' #'/sample/'
test_path = DATA_HOME_DIR + '/test/' 
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

In [ ]:
vgg = Vgg16()

In [ ]:
batch_size=64
no_of_epochs=3

In [ ]:
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size*2)
vgg.finetune(batches)
vgg.model.optimizer.lr = 0.01   # ?

In [ ]:
latest_weights_filename = None
for epoch in range(no_of_epochs):
    print "Running epoch: %d" % epoch
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weights_filename = 'ft%d.h5' % epoch
    vgg.model.save_weights(results_path+latest_weights_filename)
print "Completed %s fit operations" % no_of_epochs

In [ ]:
# 4 generate predictions

In [ ]:
batches, preds = vgg.test(test_path, batch_size = batch_size*2)

In [ ]:
print preds[:5]

filenames = batches.filenames
print filenames[:5]

In [ ]:
save_array(results_path + 'test_preds.dat', preds)
save_array(results_path + 'filenames.dat', filenames)

In [ ]:
# 5 validate the preds

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(expected_labels, our_labels)
plot_confusion_matrix(cm, val_batches.class_indices)

In [ ]:
# 6 submit

In [ ]:
preds = load_array(results_path + 'test_preds.dat')
filenames = load_array(results_path + 'filenames.dat')

In [ ]:
filenames = batches.filenames
ids = np.array([f for f in filenames])

In [ ]:
print "Raw Predictions: " + str(preds[:5])

In [ ]:
preds = preds.clip(min=0.05, max=0.95)

In [ ]:
subm = np.stack([ids,preds], axis=1)
subm[:5]

In [ ]:
%cd $DATA_HOME_DIR
submission_file_name = 'submission1.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.8f,%.8f,%.8f,%.8f,%.8f,%.8f,%.8f,%.8f', header='image,ALB, BET, DOL, LAG, NoF, OTHER, SHARK, YFT', comments='')

In [ ]:
from IPython.display import FileLink
%cd $LESSON_HOME_DIR
FileLink('data/fish/'+submission_file_name)